In [30]:
from os.path import commonprefix
from FAdo.reex import *
from FAdo.cfg import *
from FAdo.fa import *
import cProfile
%run _eTree_.ipynb

1) Gerar expressão regular aleatória

In [23]:
%time op = reStringRGenerator(Sigma={"a","b"}, size=1000, cfgr=reGrammar["g_rpn"], epsilon=None)
a1 = op.generate()
b1 = rpn2regexp(a1).reduced()
print "RegEx generated!"
b2 = b1.marked()

CPU times: user 3.69 s, sys: 15.7 ms, total: 3.7 s
Wall time: 3.7 s
RegEx generated!


2) Criar árvore e gerar FollowList da expressão e compara-la com FollowListsD do FAdo

In [24]:
b2 = b1.marked()
t = eTree(b1)
print "Tree done!"
n = t.root
l = n.First()
def compare():
    d = b2.followListsD()
    k = t.followList()
    for x in k:
        x2 = x.exp
        for z in k[x]:
            if z.exp not in set(d[x2]):
                return False
    return True
print "Equal?",compare()

Tree done!
Equal? True


3) Comparar tempos de execução de FollowList e FollowListsD

In [25]:
%time t.followList()
cProfile.run("t.followList()")
%time b2.followListsD()
cProfile.run("b2.followListsD()")

CPU times: user 935 ms, sys: 8.01 ms, total: 943 ms
Wall time: 932 ms
         947073 function calls (812363 primitive calls) in 1.186 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.075    0.075    1.186    1.186 <ipython-input-22-2160078ae33f>:12(followList)
    67516    0.041    0.000    0.041    0.000 <ipython-input-22-5bb6fbfa7e03>:100(First)
    12091    0.007    0.000    0.007    0.000 <ipython-input-22-5bb6fbfa7e03>:109(Last)
215243/80533    0.206    0.000    0.221    0.000 <ipython-input-22-5bb6fbfa7e03>:117(LSA)
    82369    0.217    0.000    1.102    0.000 <ipython-input-22-5bb6fbfa7e03>:76(follow)
    20700    0.073    0.000    0.089    0.000 <ipython-input-22-5bb6fbfa7e03>:89(follow_concat)
    46816    0.147    0.000    0.180    0.000 <ipython-input-22-5bb6fbfa7e03>:94(follow_star)
        1    0.000    0.000    1.186    1.186 <string>:1(<module>)
    82369    0.309    0.000    0.369    0.000 ge

4) Testar FollowList para o exemplo de B.Groz e S.Manneth

In [26]:
e4 = str2regexp("((c*)((ab*)(a*c)))*(ba)")
t4 = eTree(e4)
k4 = t4.followList()
for x in k4:
    print x.exp,":"
    for y in k4[x]:
        print y.exp
    print "_________"

('b', 3) :
('a', 4)
('b', 3)
('c', 5)
_________
('c', 5) :
('a', 2)
('c', 1)
('b', 6)
_________
('a', 7) :
_________
('a', 2) :
('a', 4)
('b', 3)
('c', 5)
_________
('c', 1) :
('a', 2)
('c', 1)
_________
('a', 4) :
('a', 4)
('c', 5)
_________
('b', 6) :
('a', 7)
_________


5) Testar BuildNext para exemplo anterior

In [35]:
e4 = str2regexp("((c*)((ab*)(a*c)))*(ba)")
t4 = eTree(e4)
print buildNext('b',t4.root,set())
for x in  t4.next:
    print x.exp,":"
    for y in t4.next[x]:
        print y.exp
    print "----------"

True
('b', 6) ('a', 7) :
----------
(('c', 1)* ((('a', 2) ('b', 3)*) (('a', 4)* ('c', 5))))* (('b', 6) ('a', 7)) :
----------
('b', 6) :
----------
(('c', 1)* ((('a', 2) ('b', 3)*) (('a', 4)* ('c', 5))))* :
('b', 6)
----------
('b', 3) :
----------
('b', 3)* :
----------
